In [35]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rdkit import RDLogger    
RDLogger.DisableLog('rdApp.info')
from rdkit import Chem
from rdkit.Chem.Descriptors import MolWt,MolLogP,HeavyAtomCount
from rdkit import RDConfig
from rdkit.Chem import ChemicalFeatures
from rdkit.Chem.Pharm2D.SigFactory import SigFactory
from rdkit.Chem.Pharm2D import Generate
from tqdm import tqdm
from multiprocessing import Pool

In [36]:
data_path = 'data/pubchem-1000.txt'
df_ok_path = 'data/pubchem_filter1.csv'

In [37]:
smiles = pd.read_csv(data_path,sep='\t',header=None)
smiles.columns = ['smiles']

In [38]:
def evaluate(smile):
    mol = Chem.MolFromSmiles(smile)
    if mol is None:
        return [smile, "INVALID", -999, -999, -999]
    desc_list = [MolWt(mol), MolLogP(mol),HeavyAtomCount(mol)]
    return [smile , "OK"]+desc_list

In [39]:
smiles_list = smiles['smiles'].to_list()
res = list(tqdm(map(evaluate, smiles_list),total= len(smiles_list)))

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:00<00:00, 1819.74it/s]


In [40]:
#get molecule properties
df = pd.DataFrame(res)
df = df.iloc[:,:5]

df.columns=["SMILES", "STATE","MW", "LogP","HeavyAtomCount"]
df=df[ ~ df['STATE'].str.contains('INVALID', na=False)] 
print("num_mol_succeed:",len(df))

num_mol_succeed: 1000


In [41]:
#filter molecules
df_ok = df[
    df.MW.between(*[12, 600]) & # MW
    df.LogP.between(*[-7, 5]) & #LogP
    df.HeavyAtomCount.between(*[3, 50])
    ]
print("符合要求分子数：" , len(df_ok))
df_ok.head()
df_ok.to_csv(df_ok_path,index= False,header=True)

符合要求分子数： 854


In [42]:
df_ok = pd.read_csv(df_ok_path)
df_ok

,SMILES,STATE,MW,LogP,HeavyAtomCount
0,CN(c1ccccc1)c1ccccc1C(=O)NCC1(O)CCOCC1,OK,340.423,2.72580,25
1,CC[NH+](CC)C1CCC([NH2+]C2CC2)(C(=O)[O-])C1,OK,241.355,-2.32190,17
2,COCC(CNC(=O)c1ccc2c(c1)NC(=O)C2)OC,OK,278.308,0.57240,20
3,OCCn1cc(CNc2cccc3c2CCCC3)nn1,OK,272.352,1.76130,20
4,O=C(NCc1ccc(F)cc1)N1CC=C(c2c[nH]c3ccccc23)CC1,OK,349.409,4.30590,26
...,...,...,...,...,...
849,Cc1ncnc(NCCOc2ccccc2Cl)c1C,OK,277.755,3.23774,19
850,[NH3+]C(CSc1ncccn1)c1ccc(F)c(F)c1,OK,268.312,1.83010,18
851,CC(C)Oc1cccc2[nH]cc(C(=O)C3CCCCC3)c12,OK,285.387,4.71810,21
852,COc1cccc2c1C([NH3+])(C(=O)[O-])CC(C)(C)O2,OK,251.282,-0.55660,18


In [43]:
smiles = df_ok
len(smiles)
data_split= np.array_split(smiles, 10)
len(data_split[0])

86

In [44]:
#2D pharmacophore fingerprint
def getfp(smile):
    mol = Chem.MolFromSmiles(smile)
    try:
        fp = Generate.Gen2DFingerprint(mol,sigFactory)
    except IndexError:
        pass
    else:
        fp_list = list(fp.GetOnBits())
        if fp_list ==[]:
            pass
        else:
        #fp_list=fp_list.astype(int)
            return [smile] + fp_list

fdefName = os.path.join(RDConfig.RDDataDir,'BaseFeatures.fdef')
featFactory = ChemicalFeatures.BuildFeatureFactory(fdefName)
sigFactory = SigFactory(featFactory,minPointCount=2,maxPointCount=3)
sigFactory.SetBins([(0,2),(2,5),(5,8)])
sigFactory.Init()
sigFactory.GetSigSize()

2988

In [45]:
#save the molecules with 2D pharmacophore fingerprint
for i in range(10):
    mols = data_split[i]['SMILES'].values.tolist()
    fp = list(tqdm(map(getfp,mols),total=len(data_split[i]), desc='%d 监视进度:' %(i,)))

    fp_filter_None = []
    for item in fp:
        if item != None :
            fp_filter_None.append(item)

    pharm_dir = 'data/pharm/'
    if not os.path.isdir(pharm_dir ):
                    os.makedirs(pharm_dir )

    with open('data/pharm/2d_pharm_fp_list_1_%d.txt' %i,'w') as f:
        for i in fp_filter_None:
            for j in i:
                f.write(str(j))
                f.write(' ')
            f.write('\n')
        f.close()

9 监视进度:: 100%|██████████| 85/85 [00:00<00:00, 224.28it/s]


In [46]:
# read the molecules
fp=[]
for i in range(10):
    file=open('data/pharm/2d_pharm_fp_list_1_%d.txt' %i, mode='r',encoding='UTF-8')
    contents = file.readlines()
    for msg in tqdm(contents,desc='进度： %s' %i):
        msg = msg.strip('\n')
        adm = msg.split(' ')
        fp.append(adm)
    file.close()

进度： 9: 100%|██████████| 73/73 [00:00<00:00, 73179.78it/s]


2D pharmacophore fingerprint

In [47]:
def split(sm):
    '''
    function: Split SMILES into words. Care for Cl, Br, Si, Se, Na etc.
    input: A SMILES
    output: A string with space between words
    '''
    arr = []
    i = 0
    while i < len(sm)-1:
        if not sm[i] in ['%', 'C', 'B', 'S', 'N', 'R', 'X', 'L', 'A', 'M', \
                        'T', 'Z', 's', 't', 'H', '+', '-', 'K', 'F']:
            arr.append(sm[i])
            i += 1
        elif sm[i]=='%':
            arr.append(sm[i:i+3])
            i += 3
        elif sm[i]=='C' and sm[i+1]=='l':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='C' and sm[i+1]=='a':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='C' and sm[i+1]=='u':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='B' and sm[i+1]=='r':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='B' and sm[i+1]=='e':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='B' and sm[i+1]=='a':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='B' and sm[i+1]=='i':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='S' and sm[i+1]=='i':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='S' and sm[i+1]=='e':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='S' and sm[i+1]=='r':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='N' and sm[i+1]=='a':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='N' and sm[i+1]=='i':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='R' and sm[i+1]=='b':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='R' and sm[i+1]=='a':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='X' and sm[i+1]=='e':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='L' and sm[i+1]=='i':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='A' and sm[i+1]=='l':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='A' and sm[i+1]=='s':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='A' and sm[i+1]=='g':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='A' and sm[i+1]=='u':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='M' and sm[i+1]=='g':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='M' and sm[i+1]=='n':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='T' and sm[i+1]=='e':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='Z' and sm[i+1]=='n':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='s' and sm[i+1]=='i':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='s' and sm[i+1]=='e':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='t' and sm[i+1]=='e':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='H' and sm[i+1]=='e':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='+' and sm[i+1]=='2':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='+' and sm[i+1]=='3':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='+' and sm[i+1]=='4':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='-' and sm[i+1]=='2':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='-' and sm[i+1]=='3':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='-' and sm[i+1]=='4':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='K' and sm[i+1]=='r':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='F' and sm[i+1]=='e':
            arr.append(sm[i:i+2])
            i += 2
        else:
            arr.append(sm[i])
            i += 1
    if i == len(sm)-1:
        arr.append(sm[i])
    return ' '.join(arr) 

In [48]:
pharm_corpus_path = 'data/pharm_corpus1.txt'
smi_corpus_path = 'data/smi_corpus1.txt'
smi_pharm_filepath = 'data/smi_pharm_corpus1.txt'
#from utils import split
with open(pharm_corpus_path, 'w') as f1, open(smi_corpus_path, 'w') as f2:
    for i in tqdm(range(len(fp))):
        if fp[i] is None:
            pass
        else:
            sm = fp[i][0]
            word = str(fp[i][1:])
            word  = word.replace("'", "")
            word  = word.replace(",", "")
            word  = word.replace("[", "")
            word  = word.replace("]", "")
        #word
            f1.write(word + '\n')
            f2.write(split(sm) +'\n')
print('Built pharm & smi corpus file!')

100%|██████████| 706/706 [00:00<00:00, 25281.56it/s]

Built pharm & smi corpus file!


InChI

In [49]:
def get_inchi(smile):
    mol = Chem.MolFromSmiles(smile)
    inchi = Chem.MolToInchi(mol)
    return smile +' ' + inchi

In [50]:
from tqdm import tqdm
fp=[]
with open ('smiles.txt' ,'w')as f:
    for i in range(10):
        file=open('data/pharm/2d_pharm_fp_list_1_%d.txt' %i, mode='r',encoding='UTF-8')
        contents = file.readlines()   
        for msg in tqdm(contents,desc='进度： %s' %i):
            msg = msg.strip('\n')
            adm = msg.split(' ')
            fp.append(adm[0])
            f.write(adm[0]+'\n')
        file.close()

进度： 5: 100%|██████████| 71/71 [00:00<00:00, 71225.92it/s]


进度： 9: 100%|██████████| 73/73 [00:00<?, ?it/s]


In [52]:
inchi = list(tqdm(map(get_inchi,fp),total=len(fp), desc='%d 监视进度:' ))
inchi_filter_None = []
for item in inchi:
    if item != None :
        inchi_filter_None.append(item)

    inchi_dir = 'data/inchi/'
    if not os.path.isdir(inchi_dir ):
                    os.makedirs(inchi_dir )

with open('data/inchi/inchi_list.txt','w') as f:
    for i in inchi_filter_None:
        f.write(i)
        f.write('\n')
    f.close()

%d 监视进度:: 100%|██████████| 706/706 [00:00<00:00, 2510.24it/s]


In [53]:
inchi_df = pd.read_csv('data/inchi/inchi_list.txt', sep=' ', header = None)
inchi_df

,0,1
0,CN(c1ccccc1)c1ccccc1C(=O)NCC1(O)CCOCC1,InChI=1S/C20H24N2O3/c1-22(16-7-3-2-4-8-16)18-1...
1,CC[NH+](CC)C1CCC([NH2+]C2CC2)(C(=O)[O-])C1,InChI=1S/C13H24N2O2/c1-3-15(4-2)11-7-8-13(9-11...
2,COCC(CNC(=O)c1ccc2c(c1)NC(=O)C2)OC,InChI=1S/C14H18N2O4/c1-19-8-11(20-2)7-15-14(18...
3,OCCn1cc(CNc2cccc3c2CCCC3)nn1,InChI=1S/C15H20N4O/c20-9-8-19-11-13(17-18-19)1...
4,O=C(NCc1ccc(F)cc1)N1CC=C(c2c[nH]c3ccccc23)CC1,InChI=1S/C21H20FN3O/c22-17-7-5-15(6-8-17)13-24...
...,...,...
701,Cc1ncnc(NCCOc2ccccc2Cl)c1C,InChI=1S/C14H16ClN3O/c1-10-11(2)17-9-18-14(10)...
702,[NH3+]C(CSc1ncccn1)c1ccc(F)c(F)c1,InChI=1S/C12H11F2N3S/c13-9-3-2-8(6-10(9)14)11(...
703,CC(C)Oc1cccc2[nH]cc(C(=O)C3CCCCC3)c12,InChI=1S/C18H23NO2/c1-12(2)21-16-10-6-9-15-17(...
704,COc1cccc2c1C([NH3+])(C(=O)[O-])CC(C)(C)O2,"InChI=1S/C13H17NO4/c1-12(2)7-13(14,11(15)16)10..."


In [54]:
def split_inchi(inchi):
    '''
    function: Split inchi into words. Care for Cl, Br, Si, Se, Na etc.
    input: A inchi
    output: A string with space between words
    '''
    arr = []
    i = 0
    while i < len(inchi)-1:
        if not inchi[i] in ['%', 'C', 'B', 'S', 'N', 'R', 'X', 'L', 'A', 'M', \
                        'T', 'Z', 's', 't', 'H', '+', 'K', 'F','I','1','2','3','4']:
            arr.append(inchi[i])
            i += 1
        elif inchi[i]=='%':
            arr.append(inchi[i:i+3])
            i += 3
        elif inchi[i]=='C' and inchi[i+1]=='l':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='C' and inchi[i+1]=='a':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='C' and inchi[i+1]=='u':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='B' and inchi[i+1]=='r':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='B' and inchi[i+1]=='e':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='B' and inchi[i+1]=='a':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='B' and inchi[i+1]=='i':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='S' and inchi[i+1]=='i':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='S' and inchi[i+1]=='e':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='S' and inchi[i+1]=='r':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='N' and inchi[i+1]=='a':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='N' and inchi[i+1]=='i':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='R' and inchi[i+1]=='b':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='R' and inchi[i+1]=='a':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='X' and inchi[i+1]=='e':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='L' and inchi[i+1]=='i':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='A' and inchi[i+1]=='l':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='A' and inchi[i+1]=='s':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='A' and inchi[i+1]=='g':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='A' and inchi[i+1]=='u':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='M' and inchi[i+1]=='g':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='M' and inchi[i+1]=='n':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='T' and inchi[i+1]=='e':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='Z' and inchi[i+1]=='n':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='s' and inchi[i+1]=='i':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='s' and inchi[i+1]=='e':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='t' and inchi[i+1]=='e':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='H' and inchi[i+1]=='e':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='+' and inchi[i+1]=='2':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='+' and inchi[i+1]=='3':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='+' and inchi[i+1]=='4':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='K' and inchi[i+1]=='r':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='F' and inchi[i+1]=='e':
            arr.append(inchi[i:i+2])
            i += 2
        elif inchi[i]=='I' and inchi[i+1]=='n' and inchi[i+2]=='C':
            arr.append(inchi[i:i+9])
            i += 9
        elif inchi[i]=='1' and (inchi[i+1]=='0'or inchi[i+1]=='1' or inchi[i+1]=='2' or inchi[i+1]=='3'or inchi[i+1]=='4'or inchi[i+1]=='5'or inchi[i+1]=='6'or inchi[i+1]=='7'or inchi[i+1]=='8'or inchi[i+1]=='9'):
            arr.append(inchi[i:i+2])
            i += 2   
        elif inchi[i]=='2' and (inchi[i+1]=='0'or inchi[i+1]=='1' or inchi[i+1]=='2' or inchi[i+1]=='3'or inchi[i+1]=='4'or inchi[i+1]=='5'or inchi[i+1]=='6'or inchi[i+1]=='7'or inchi[i+1]=='8'or inchi[i+1]=='9'):
            arr.append(inchi[i:i+2])
            i += 2   
        elif inchi[i]=='3' and (inchi[i+1]=='0'or inchi[i+1]=='1' or inchi[i+1]=='2' or inchi[i+1]=='3'or inchi[i+1]=='4'or inchi[i+1]=='5'or inchi[i+1]=='6'or inchi[i+1]=='7'or inchi[i+1]=='8'or inchi[i+1]=='9'):
            arr.append(inchi[i:i+2])
            i += 2 
        elif inchi[i]=='4' and (inchi[i+1]=='0'or inchi[i+1]=='1' or inchi[i+1]=='2' or inchi[i+1]=='3'or inchi[i+1]=='4'or inchi[i+1]=='5'or inchi[i+1]=='6'or inchi[i+1]=='7'or inchi[i+1]=='8'or inchi[i+1]=='9'):
            arr.append(inchi[i:i+2])
            i += 2 
        else: 
            arr.append(inchi[i])
            i += 1
    if i == len(inchi)-1:
        arr.append(inchi[i])
    return ' '.join(arr) 

In [55]:
inchi = []
with open('data/inchi/inchi_list.txt','r') as file:
    contents = file.readlines()
    for msg in tqdm(contents):
        msg = msg.strip('\n')
        adm = msg.split(' ')
        inchi.append(adm)



100%|██████████| 706/706 [00:00<00:00, 707569.56it/s]


In [56]:
inchi_corpus_path = 'data/inchi_corpus1.txt'
smi_corpus_path = 'data/smi_corpus2.txt'
smi_inchi_filepath = 'data/smi_inchi_corpus1.txt'
#from utils import split
with open(inchi_corpus_path, 'w') as f1, open(smi_corpus_path, 'w') as f2:
    for i in tqdm(range(len(inchi))):
        if inchi[i] is None:
            pass
        else:
            sm = inchi[i][0]
            word = inchi[i][1]
            f1.write(split_inchi(word) + '\n')
            f2.write(split(sm) +'\n')
print('Built inchi & smi corpus file!')

  0%|          | 0/706 [00:00<?, ?it/s]

100%|██████████| 706/706 [00:00<00:00, 7694.13it/s]

Built inchi & smi corpus file!


PubChem

In [57]:
from pubchemfp import GetPubChemFPs
import numpy as np
from rdkit import Chem
from tqdm import tqdm

In [58]:
def GetPubChemFpBits(smiles):
    mol = Chem.MolFromSmiles(smiles)
    mol2 = Chem.AddHs(mol)
    try:
        result = GetPubChemFPs(mol2)
    except:
        pass
    else:
        bit = []
        for i in range(881):
            if result[i] == True:
                bit.append(i)
        return [smiles] + bit

In [59]:

fp=[]
with open ('smiles.txt' ,'w')as f:
    for i in range(10):
        file=open('data/pharm/2d_pharm_fp_list_1_%d.txt' %i, mode='r',encoding='UTF-8')
        contents = file.readlines()   
        for msg in tqdm(contents,desc='进度： %s' %i):
            msg = msg.strip('\n')
            adm = msg.split(' ')
            fp.append(adm[0])
            f.write(adm[0]+'\n')
        file.close()

进度： 9: 100%|██████████| 73/73 [00:00<00:00, 73214.78it/s]


In [60]:
smiles = fp
len(smiles)
data_split= np.array_split(smiles, 10)
len(data_split[0])

71

In [61]:
for i in range(10):
    mols = data_split[i].tolist()
    fp = list(tqdm(map(GetPubChemFpBits,mols),total=len(data_split[i]), desc='%d 监视进度:' %(i,)))

    fp_filter_None = []
    for item in fp:
        if item != None :
            fp_filter_None.append(item)

    pubchem_dir = 'data/pubchem/'
    if not os.path.isdir(pubchem_dir ):
                    os.makedirs(pubchem_dir )

    with open('data/pubchem/pubchem_fp_list_%d.txt' %i,'w') as f:
        for i in fp_filter_None:
            for j in i:
                f.write(str(j))
                f.write(' ')
            f.write('\n')
        f.close()

9 监视进度:: 100%|██████████| 70/70 [00:00<00:00, 171.51it/s]


In [62]:
pub_fp=[]
for i in range(10):
    file=open('data/pubchem/pubchem_fp_list_%d.txt' %i, mode='r',encoding='UTF-8')
    contents = file.readlines()
    for msg in tqdm(contents,desc='进度： %s' %i):
        msg = msg.strip('\n')
        adm = msg.split(' ')
        pub_fp.append(adm)
    file.close()

进度： 9: 100%|██████████| 70/70 [00:00<00:00, 70189.17it/s]


In [68]:
pubchem_corpus_path = 'data/pubchemfp_corpus1.txt'
smi_corpus_path = 'data/smi_corpus3.txt'
smi_pubchem_filepath = 'data/smi_pubchemfp_corpus1.txt'
#from utils import split
with open(pubchem_corpus_path, 'w') as f1, open(smi_corpus_path, 'w') as f2:
    for i in tqdm(range(len(pub_fp))):
        if pub_fp[i] is None:
            pass
        else:
            sm = pub_fp[i][0]
            word = str(pub_fp[i][1:])
            word  = word.replace("'", "")
            word  = word.replace(",", "")
            word  = word.replace("[", "")
            word  = word.replace("]", "")
        #word
            f1.write(word + '\n')
            f2.write(split(sm) +'\n')
print('Built pharm & smi corpus file!')

100%|██████████| 706/706 [00:00<00:00, 22119.97it/s]

Built pharm & smi corpus file!


Build smi_pharm_inchi_pubchem_corpus

In [69]:
pharm_corpus_path = 'data/pharm_corpus1.txt'
smi_corpus_path = 'data/smi_corpus1.txt'
smi_pharm_filepath = 'data/smi_pharm_inchi_pubchem_corpus2.txt'
inchi_corpus_path = 'data/inchi_corpus1.txt'
pubchem_corpus_path = 'data/pubchemfp_corpus1.txt'

s = []
f = open(smi_corpus_path,'r')
for line in tqdm(f):
    line = line.strip('\n')
    s.append(line)
f.close()

pubfp = []
f = open(pubchem_corpus_path,'r')
for line in tqdm(f):
    line = line.strip('\n')
    pubfp.append(line)
f.close()

fp = []
f = open(pharm_corpus_path,'r')
for line in tqdm(f):
    line = line.strip('\n')
    fp.append(line)
f.close()

inchi = []
f = open(inchi_corpus_path,'r')
for line in tqdm(f):
    line = line.strip('\n')
    inchi.append(line)
f.close()

dfs = pd.DataFrame(s)
dffp = pd.DataFrame(fp)
dfinchi = pd.DataFrame(inchi)
dfpub = pd.DataFrame(pubfp)

data = pd.concat([dfs, dffp,dfinchi,dfpub],axis=1,ignore_index=True)
data.columns = ['smiles','pharmfp','inchi','pubchemfp']
data.to_csv(smi_pharm_filepath,index=False,sep='\t',header=False)
print('Built smi_pharm_inchi translation corpus file!')

706it [00:00, 708924.74it/s]
706it [00:00, 235912.89it/s]
706it [00:00, 354080.91it/s]
706it [00:00, 354292.73it/s]

Built smi_pharm_inchi translation corpus file!


In [70]:
import BuildVocab

pharm_vocab_path = 'data/pharm_vocab1.pkl'
smi_vocab_path = 'data/smi_vocab1.pkl'

BuildVocab.built_vocab(pharm_corpus_path,pharm_vocab_path)
BuildVocab.built_vocab(smi_corpus_path,smi_vocab_path)

Building Vocab
VOCAB SIZE: 1396
Vocab build done! 
Building Vocab
VOCAB SIZE: 37
Vocab build done! 


In [71]:
inchi_vocab_path = 'data/inchi_vocab1.pkl'
inchi_corpus_path = 'data/inchi_corpus1.txt'
BuildVocab.built_vocab(inchi_corpus_path,inchi_vocab_path)

Building Vocab
VOCAB SIZE: 73
Vocab build done! 


In [72]:

pubfp_vocab_path = 'data/pubfp_vocab1.pkl'
pubfp_corpus_path = 'data/pubchemfp_corpus1.txt'
BuildVocab.built_vocab(pubfp_corpus_path,pubfp_vocab_path)

Building Vocab
VOCAB SIZE: 646
Vocab build done! 
